<a href="https://colab.research.google.com/github/sergekamanzi/Nithub/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install beautifulsoup4 requests
!pip install textblob
!pip install scikit-learn flask

In [3]:
import requests
from bs4 import BeautifulSoup
import re

# Function to scrape article
def scrape_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = [p.text for p in soup.find_all('p')]
    return ' '.join(paragraphs)

# URL of the news article (you can change this to any news website)
url = 'https://www.bbc.com/news/world-60525350'
article_text = scrape_article(url)

# Split the article text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article_text)
print(sentences[:10])  # Show the first 10 sentences

# Limit to 200 sentences
sentences = sentences[:200]


['Sir Keir Starmer said securing a lasting peace in Ukraine was "essential" to deter Vladimir Putin from future aggression.', " Europe's hastily convened security summit in Paris is proof of leaders' anxiety about their role in defending Ukraine, the BBC's Katya Adler writes.", "It comes as Donald Trump's envoy says Europe will not be involved in peace talks to end the war.", " There has been no increase in radiation levels at the plant, Ukraine's president says.", 'How are Russia and Ukraine placed after three years of fighting and with Donald Trump suggesting talks on ending the war could begin soon?', "Europe's hastily convened security summit in Paris is proof of leaders' anxiety about their role in defending Ukraine, the BBC's Katya Adler writes.", 'Poland\'s PM said if Europe fails "to spend big on defence now, we will be forced to spend 10 times more" during war.', 'Different narratives from senior US officials on the future of Ukraine have sewn confusion in Europe?', "It comes 

In [4]:
from textblob import TextBlob

# Function to get sentiment
def annotate_sentiment(sentence):
    analysis = TextBlob(sentence)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

# Annotate all sentences
annotated_sentences = [(sentence, annotate_sentiment(sentence)) for sentence in sentences]

# Show the first 10 annotated sentences
for sentence, sentiment in annotated_sentences[:10]:
    print(f'{sentence} -> {sentiment}')

# Save as a CSV file
import pandas as pd

df = pd.DataFrame(annotated_sentences, columns=['Sentence', 'Sentiment'])
df.to_csv('annotated_sentences.csv', index=False)


Sir Keir Starmer said securing a lasting peace in Ukraine was "essential" to deter Vladimir Putin from future aggression. -> Neutral
 Europe's hastily convened security summit in Paris is proof of leaders' anxiety about their role in defending Ukraine, the BBC's Katya Adler writes. -> Neutral
It comes as Donald Trump's envoy says Europe will not be involved in peace talks to end the war. -> Neutral
 There has been no increase in radiation levels at the plant, Ukraine's president says. -> Neutral
How are Russia and Ukraine placed after three years of fighting and with Donald Trump suggesting talks on ending the war could begin soon? -> Neutral
Europe's hastily convened security summit in Paris is proof of leaders' anxiety about their role in defending Ukraine, the BBC's Katya Adler writes. -> Neutral
Poland's PM said if Europe fails "to spend big on defence now, we will be forced to spend 10 times more" during war. -> Negative
Different narratives from senior US officials on the future 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

# Load the annotated data
df = pd.read_csv('annotated_sentences.csv')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Sentence'], df['Sentiment'], test_size=0.2, random_state=42)

# Convert text data to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Save the model and vectorizer
pickle.dump(model, open('sentiment_model.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

# Evaluate model performance
print(f'Model accuracy: {model.score(X_test_tfidf, y_test)}')


Model accuracy: 0.7142857142857143


In [11]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the pre-trained model and vectorizer
model = pickle.load(open('sentiment_model.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

# Function to predict sentiment
def predict_sentiment(sentence):
    # Transform the input sentence to TF-IDF features
    sentence_tfidf = vectorizer.transform([sentence])

    # Predict the sentiment using the pre-trained model
    prediction = model.predict(sentence_tfidf)[0]

    return prediction

# Get input from the user
user_input = input("Enter a sentence to predict its sentiment: ")

# Predict and print the sentiment
predicted_sentiment = predict_sentiment(user_input)
print(f'Sentiment: {predicted_sentiment}')


Enter a sentence to predict its sentiment: hey what up
Sentiment: Neutral
